In [1]:
import random
from numpy.random import choice
import numpy as np

"""
p: num_ants (100 or 10), index usually denoted by n
e: pheromone_evaporation_rate (0.9 or 0.6)
b: num_bins (10 or 50), depends or BPP1 or BPP2, index usually denoted by j
k: num_items (500), index usually denoted by i
w_b: list_weight_in_bins
w_i: #list_item_weight (=item no. i or =(i^2)/2)
bpp: BPP1 or BPP2 (1 or 2)
Pij: corresponding probabilities to each entry in T
T: Pheromone matrix
SE: set of ant paths from S to E (item in list ranges from 1 to 10, representing the bins)
"""

def GenerateSE(T, k, b, p):
    SE = [[] for n in range(p)] 
    denom=np.sum(T, axis = 1) # denominator
    Pij=[T[i]/denom[i] for i in range(k)] # probabilities
    for n in range(p):
        for i in range(k):
            SE[n].append(choice([j for j in range(b)], p=Pij[i])+1) # using choice to add selected bin acc. to prob.
    return SE

def Fitness(SE, k, b, p, w_i):
    d = [0 for n in range(p)] # difference d between the heaviest and lightest bins
    for n in range(p):
        w_b=[0 for j in range(b)]
        for i in range(k):
            w_b[SE[n][i]-1] = w_b[SE[n][i]-1] + w_i[SE[n][i]-1]
        d[n] = max(w_b) - min(w_b)
    return d

def GenerateResults(d, SE):
    results = [min(d), SE[d.index(min(d))], max(d), SE[d.index(max(d))], sum(d) / len(d)]
    return results

def PrintResults(d, SE):
    results = GenerateResults(d, SE)
    print("Fitness:    Best: ", results[0], "  Worst: ", results[2], "  Average: ", results[4])
    print("    Best path: ", results[1])
    print("    Worst path: ", results[3])
    return None

In [2]:
def ACO_BPP(p, e, bpp):

    # Initialization
    iteration = int(50000/p)
    k = 500
    if (bpp == 1):
        b = 10
        w_i = [i + 1 for i in range(k)]
    elif (bpp == 2):
        b = 50
        w_i=[((i + 1) * (i +1 )) / 2 for i in range(k)]
        
    # 1. Randomly distribute small amounts of pheromone (between 0 and 1) on the construction graph.
    T=[[random.random() for j in range(b)] for i in range(k)]
    
    # 5. Until termination, repeat
    for itr in range(iteration):
        
        # 2. Generate a set of p ant paths from S to E
        SE = GenerateSE(T, k, b, p)
        
        #3. Update the pheromone in pheromone table.
        d = Fitness(SE, k, b, p, w_i)
        delta = 100/min(d) #100/fitness
        for i in range(k):
            T[i][SE[d.index(min(d))][i]-1] += delta #modified to only update the path with best fitness
        
        # 4. Evaporate the pheromone for all links in the graph
        T = [[e * T[i][j] for j in range(b)] for i in range(k)] # evaporation: e * T (pheromone matrix)

        # Printing results, best & worst fitness of first & last generation
        if (itr == 0):
            print("First generation: ")
            PrintResults(d, SE)     
        elif (itr == iteration - 1):
            print("Last Generation: ")
            PrintResults(d, SE)
    
    return None

# Problem 1

For problem 1, there are 10 bins, 500 items. The weight of items are 1,2,...,500. 

The algorithm aims at utilizing p number of ants and 50000/p iterations to find an optimal ant path for putting items of different weights into a certain number of bins such that the maximum weight difference of heaviest and lightest bin is kept to a minimum.

The algorithm prints the fitness 'Best', 'Worst' & 'Avereage'of the first and last generation, and also the best path and worst path of first and last generation. Note: a small value in fitness indicates a better path.


1a) Trial of ACO with p = 200 and e = 0.9

In [3]:
ACO_BPP(200, 0.9, 1)

First generation: 
Fitness:    Best:  326   Worst:  613   Average:  454.49
    Best path:  [6, 3, 10, 6, 4, 6, 6, 1, 5, 2, 8, 7, 3, 9, 1, 8, 1, 7, 3, 10, 4, 8, 7, 4, 6, 3, 1, 1, 9, 1, 2, 10, 6, 6, 1, 8, 10, 2, 9, 3, 1, 5, 3, 8, 1, 6, 5, 9, 8, 9, 2, 7, 2, 4, 1, 3, 1, 9, 7, 5, 2, 4, 7, 7, 1, 5, 6, 4, 1, 2, 1, 2, 7, 8, 8, 7, 5, 1, 8, 2, 9, 9, 1, 8, 9, 10, 7, 3, 6, 5, 3, 7, 8, 1, 9, 2, 2, 2, 5, 3, 2, 8, 8, 2, 1, 1, 2, 8, 7, 3, 3, 5, 6, 7, 4, 3, 1, 3, 4, 6, 9, 2, 5, 1, 2, 1, 10, 3, 3, 6, 6, 10, 8, 4, 6, 1, 5, 5, 1, 1, 2, 1, 6, 3, 3, 5, 4, 5, 4, 2, 8, 2, 6, 3, 1, 6, 9, 3, 1, 9, 7, 9, 5, 10, 3, 6, 7, 6, 7, 3, 5, 10, 10, 9, 8, 1, 10, 1, 10, 3, 2, 5, 2, 5, 8, 3, 9, 7, 10, 3, 9, 8, 9, 7, 7, 2, 3, 3, 9, 5, 8, 9, 7, 8, 3, 6, 1, 7, 7, 2, 4, 1, 6, 3, 2, 4, 5, 7, 4, 6, 2, 3, 1, 9, 10, 6, 4, 6, 4, 4, 9, 5, 5, 9, 5, 6, 10, 1, 8, 4, 9, 3, 3, 3, 6, 2, 4, 9, 9, 7, 8, 7, 3, 7, 6, 9, 5, 7, 8, 3, 6, 7, 9, 1, 9, 10, 9, 2, 5, 5, 3, 6, 4, 2, 4, 9, 10, 6, 5, 5, 3, 1, 6, 4, 1, 4, 7, 6, 6, 3, 10, 7, 8, 3, 1, 8, 3,

1b) Trial of the ACO with p = 200 and e = 0.65

In [4]:
ACO_BPP(200, 0.65, 1)

First generation: 
Fitness:    Best:  359   Worst:  713   Average:  475.85
    Best path:  [9, 7, 8, 6, 3, 4, 3, 7, 6, 2, 10, 10, 8, 7, 7, 4, 1, 5, 1, 5, 8, 2, 2, 9, 1, 9, 1, 7, 10, 3, 1, 2, 3, 10, 8, 7, 7, 2, 2, 7, 4, 9, 5, 8, 6, 3, 3, 1, 7, 3, 6, 3, 2, 6, 3, 4, 9, 2, 2, 4, 4, 9, 7, 10, 6, 10, 8, 4, 4, 8, 9, 4, 6, 6, 4, 9, 7, 2, 9, 2, 5, 2, 4, 6, 4, 9, 3, 3, 10, 4, 5, 3, 4, 1, 8, 6, 2, 5, 2, 7, 6, 6, 5, 1, 8, 10, 5, 8, 8, 5, 5, 2, 8, 6, 5, 1, 1, 8, 6, 4, 6, 7, 2, 6, 2, 5, 8, 6, 9, 7, 5, 8, 7, 5, 1, 5, 1, 5, 6, 2, 1, 7, 5, 1, 10, 6, 7, 4, 8, 2, 5, 3, 7, 5, 1, 2, 10, 8, 7, 5, 4, 8, 2, 6, 8, 8, 6, 5, 2, 7, 7, 5, 10, 1, 4, 5, 1, 3, 3, 6, 1, 9, 1, 2, 8, 7, 4, 5, 4, 10, 8, 4, 1, 3, 10, 1, 6, 7, 8, 6, 7, 1, 9, 6, 2, 5, 3, 10, 9, 3, 2, 8, 10, 1, 4, 3, 5, 8, 9, 8, 7, 5, 4, 10, 9, 1, 6, 2, 6, 5, 5, 1, 1, 7, 2, 3, 9, 8, 1, 6, 1, 5, 1, 8, 1, 2, 8, 2, 7, 5, 9, 10, 10, 7, 4, 6, 6, 3, 1, 2, 9, 6, 9, 10, 1, 7, 9, 6, 4, 3, 10, 5, 9, 5, 6, 2, 8, 5, 8, 7, 3, 3, 5, 9, 10, 6, 10, 2, 5, 3, 6, 6, 3, 2, 10, 

1c) Trial of ACO with p = 200 and e = 0.4

In [5]:
ACO_BPP(200, 0.4, 1)

First generation: 
Fitness:    Best:  346   Worst:  585   Average:  459.565
    Best path:  [9, 5, 5, 6, 5, 10, 5, 2, 6, 6, 7, 1, 10, 5, 9, 9, 3, 7, 1, 2, 1, 1, 3, 9, 7, 8, 7, 1, 1, 2, 10, 1, 10, 9, 9, 9, 5, 2, 1, 1, 5, 1, 7, 8, 10, 5, 9, 5, 8, 8, 4, 5, 10, 1, 7, 4, 8, 8, 7, 1, 1, 2, 9, 4, 8, 8, 5, 1, 3, 1, 4, 3, 8, 7, 7, 8, 5, 5, 7, 6, 6, 8, 6, 4, 9, 7, 2, 4, 10, 4, 8, 4, 2, 6, 3, 6, 8, 8, 6, 8, 7, 9, 9, 1, 6, 8, 3, 4, 7, 3, 7, 5, 7, 3, 1, 4, 7, 10, 3, 4, 1, 1, 9, 6, 4, 5, 5, 1, 9, 5, 1, 2, 1, 3, 4, 6, 7, 9, 6, 4, 1, 10, 5, 8, 3, 9, 10, 3, 6, 2, 1, 1, 7, 5, 6, 9, 8, 1, 9, 4, 3, 1, 6, 4, 6, 4, 4, 6, 2, 5, 3, 7, 4, 4, 4, 4, 8, 5, 9, 7, 2, 9, 2, 8, 4, 7, 1, 8, 1, 4, 5, 7, 2, 10, 5, 10, 6, 10, 2, 3, 6, 4, 7, 7, 9, 5, 10, 5, 5, 4, 4, 6, 2, 9, 5, 7, 6, 10, 2, 6, 2, 2, 4, 3, 2, 5, 5, 2, 2, 10, 9, 6, 3, 10, 4, 8, 5, 9, 5, 9, 8, 2, 7, 1, 1, 8, 8, 5, 2, 7, 5, 4, 7, 6, 4, 10, 9, 5, 4, 8, 3, 3, 4, 1, 4, 1, 3, 2, 5, 6, 4, 7, 1, 3, 2, 3, 5, 1, 5, 5, 10, 2, 3, 7, 8, 5, 10, 4, 7, 6, 1, 5, 10, 10, 8, 

1d) Trial of ACO with p = 200 and e = 0.15

In [6]:
ACO_BPP(200, 0.15, 1)

First generation: 
Fitness:    Best:  356   Worst:  647   Average:  471.945
    Best path:  [7, 4, 3, 2, 6, 10, 2, 6, 6, 1, 4, 1, 9, 10, 5, 3, 3, 1, 7, 10, 10, 8, 4, 2, 5, 8, 9, 4, 3, 2, 1, 3, 10, 5, 4, 1, 2, 9, 4, 1, 7, 5, 3, 10, 3, 7, 2, 10, 10, 4, 6, 10, 5, 4, 9, 8, 8, 9, 10, 2, 3, 7, 5, 4, 5, 9, 6, 9, 2, 7, 8, 4, 3, 2, 2, 8, 3, 4, 6, 3, 3, 9, 5, 10, 9, 6, 10, 7, 7, 9, 9, 1, 2, 9, 4, 7, 9, 9, 2, 10, 6, 9, 9, 5, 7, 3, 10, 6, 3, 3, 3, 1, 8, 4, 1, 4, 9, 7, 6, 1, 5, 2, 4, 3, 5, 2, 8, 6, 3, 9, 10, 7, 4, 5, 7, 6, 9, 5, 5, 3, 9, 9, 7, 6, 7, 1, 1, 3, 10, 3, 10, 9, 8, 8, 1, 6, 5, 9, 4, 7, 1, 2, 4, 9, 4, 3, 6, 2, 7, 8, 6, 6, 1, 5, 6, 5, 4, 10, 4, 2, 9, 4, 8, 6, 5, 5, 6, 3, 3, 1, 7, 3, 8, 5, 3, 6, 1, 4, 4, 4, 4, 1, 8, 7, 7, 4, 7, 2, 3, 2, 2, 4, 8, 3, 8, 6, 8, 8, 10, 9, 10, 9, 1, 5, 2, 5, 8, 2, 2, 3, 6, 6, 7, 2, 5, 4, 8, 3, 6, 5, 10, 3, 7, 3, 5, 1, 4, 6, 1, 2, 4, 3, 5, 3, 6, 3, 1, 8, 9, 5, 3, 2, 7, 5, 8, 9, 1, 1, 3, 10, 3, 8, 6, 4, 2, 7, 2, 8, 8, 8, 8, 3, 3, 7, 2, 9, 8, 5, 1, 9, 6, 10, 5, 9, 1,

2a) Trial of ACO with p = 150 and e = 0.9

In [7]:
ACO_BPP(150, 0.9, 1)

First generation: 
Fitness:    Best:  349   Worst:  659   Average:  471.44
    Best path:  [7, 4, 10, 10, 4, 1, 9, 8, 9, 10, 2, 10, 6, 4, 4, 7, 5, 2, 10, 6, 7, 3, 7, 8, 10, 3, 8, 7, 8, 3, 7, 2, 4, 4, 9, 7, 1, 3, 1, 3, 1, 6, 5, 4, 1, 10, 5, 1, 3, 6, 3, 6, 1, 8, 4, 4, 4, 10, 10, 2, 2, 7, 8, 7, 4, 1, 3, 3, 10, 7, 3, 6, 6, 7, 1, 7, 3, 6, 4, 3, 6, 7, 6, 4, 5, 4, 6, 10, 2, 3, 1, 3, 7, 7, 2, 2, 5, 3, 3, 6, 3, 4, 10, 7, 9, 7, 5, 3, 5, 6, 4, 5, 9, 10, 7, 4, 8, 9, 4, 7, 2, 2, 5, 7, 1, 5, 10, 1, 3, 7, 3, 4, 2, 7, 3, 9, 4, 8, 1, 3, 3, 1, 3, 4, 5, 5, 5, 2, 9, 2, 4, 4, 1, 4, 7, 5, 10, 2, 3, 5, 8, 6, 2, 7, 1, 2, 7, 2, 5, 8, 9, 6, 8, 10, 1, 9, 1, 3, 3, 2, 5, 2, 4, 2, 1, 10, 7, 6, 9, 2, 3, 4, 6, 9, 5, 1, 4, 8, 3, 7, 6, 4, 10, 3, 10, 2, 7, 5, 3, 2, 4, 4, 10, 10, 1, 9, 8, 10, 8, 1, 3, 6, 9, 5, 3, 6, 2, 9, 2, 5, 5, 6, 6, 1, 6, 6, 2, 7, 8, 7, 8, 8, 1, 3, 9, 1, 5, 8, 8, 1, 8, 9, 2, 9, 2, 7, 8, 7, 6, 8, 10, 4, 5, 4, 8, 1, 1, 9, 1, 8, 1, 5, 7, 5, 2, 8, 10, 9, 7, 4, 8, 4, 4, 1, 4, 5, 3, 7, 5, 2, 3, 6, 9, 2, 3,

2b) Trial of the ACO with p = 150 and e = 0.65

In [8]:
ACO_BPP(150, 0.65, 1)

First generation: 
Fitness:    Best:  374   Worst:  727   Average:  468.1533333333333
    Best path:  [3, 10, 10, 8, 1, 3, 2, 4, 3, 4, 8, 9, 4, 1, 1, 10, 10, 6, 2, 7, 4, 6, 6, 7, 7, 10, 5, 2, 1, 2, 2, 8, 8, 5, 3, 3, 8, 3, 2, 2, 3, 5, 7, 8, 3, 5, 4, 3, 10, 4, 10, 7, 1, 7, 8, 7, 10, 1, 10, 9, 6, 3, 5, 10, 2, 9, 9, 1, 1, 10, 4, 3, 1, 8, 6, 9, 2, 8, 9, 10, 1, 7, 3, 5, 1, 4, 2, 5, 2, 4, 2, 8, 4, 7, 3, 7, 10, 4, 8, 9, 6, 10, 10, 8, 4, 6, 2, 5, 10, 5, 5, 4, 8, 5, 9, 1, 7, 2, 6, 10, 8, 4, 8, 9, 2, 3, 7, 8, 7, 10, 9, 1, 4, 6, 1, 3, 7, 3, 4, 5, 4, 6, 1, 5, 9, 1, 9, 4, 2, 5, 4, 3, 2, 2, 4, 3, 7, 10, 9, 3, 3, 1, 2, 1, 9, 1, 1, 6, 1, 3, 4, 6, 8, 7, 5, 5, 2, 1, 10, 8, 6, 1, 5, 2, 6, 7, 9, 5, 7, 3, 3, 8, 6, 9, 6, 10, 6, 1, 5, 3, 9, 2, 10, 2, 3, 4, 1, 2, 3, 10, 2, 6, 1, 1, 8, 7, 3, 7, 8, 6, 5, 6, 2, 4, 1, 7, 10, 9, 6, 3, 10, 7, 1, 5, 5, 7, 5, 9, 2, 1, 4, 8, 7, 6, 3, 8, 10, 8, 8, 3, 5, 9, 9, 1, 3, 1, 4, 7, 1, 5, 4, 9, 3, 6, 4, 9, 3, 5, 2, 7, 8, 2, 8, 6, 4, 5, 6, 8, 3, 7, 1, 4, 1, 5, 5, 8, 1, 7, 1, 4, 3

2c) Trial of ACO with p = 150 and e = 0.4

In [9]:
ACO_BPP(150, 0.4, 1)

First generation: 
Fitness:    Best:  348   Worst:  647   Average:  469.58666666666664
    Best path:  [5, 9, 7, 2, 2, 1, 5, 2, 3, 6, 9, 7, 9, 3, 5, 8, 10, 1, 3, 9, 5, 6, 5, 6, 4, 6, 3, 5, 8, 9, 9, 3, 2, 1, 2, 6, 2, 8, 7, 6, 8, 6, 7, 3, 4, 8, 10, 7, 6, 6, 5, 3, 6, 2, 10, 8, 2, 3, 1, 2, 2, 4, 9, 7, 1, 1, 9, 6, 10, 2, 7, 2, 2, 1, 6, 4, 10, 3, 10, 10, 2, 3, 2, 2, 1, 5, 5, 5, 8, 6, 6, 4, 7, 5, 7, 2, 3, 8, 5, 7, 1, 8, 3, 4, 4, 4, 3, 3, 7, 7, 1, 1, 10, 6, 8, 1, 1, 10, 6, 2, 5, 7, 2, 9, 3, 9, 8, 8, 3, 3, 8, 2, 4, 1, 3, 8, 6, 10, 4, 4, 7, 3, 6, 4, 9, 9, 6, 10, 5, 2, 9, 8, 9, 6, 7, 4, 10, 3, 1, 5, 1, 1, 7, 6, 9, 5, 5, 2, 1, 10, 8, 9, 8, 3, 8, 6, 5, 1, 4, 4, 2, 7, 6, 9, 4, 2, 8, 4, 3, 7, 6, 5, 6, 1, 3, 1, 7, 4, 8, 6, 3, 1, 5, 4, 10, 7, 3, 8, 6, 7, 2, 5, 7, 10, 2, 3, 4, 6, 9, 7, 5, 8, 5, 5, 7, 8, 1, 7, 8, 5, 2, 3, 8, 7, 9, 1, 9, 4, 8, 6, 1, 4, 2, 9, 10, 6, 1, 7, 1, 1, 9, 9, 3, 1, 2, 3, 1, 5, 2, 6, 3, 10, 4, 6, 4, 4, 1, 4, 8, 6, 2, 10, 4, 7, 1, 4, 4, 7, 9, 10, 6, 10, 3, 2, 8, 5, 5, 7, 2, 9, 6, 2, 

2d) Trial of ACO with p = 150 and e = 0.15

In [10]:
ACO_BPP(150, 0.15, 1)

First generation: 
Fitness:    Best:  356   Worst:  617   Average:  454.66
    Best path:  [7, 6, 9, 7, 10, 8, 1, 1, 6, 9, 3, 4, 8, 4, 5, 1, 10, 3, 5, 2, 7, 1, 2, 3, 10, 3, 6, 4, 10, 5, 6, 7, 10, 3, 1, 9, 8, 3, 7, 6, 2, 6, 4, 9, 9, 6, 6, 5, 4, 1, 3, 4, 5, 10, 10, 8, 2, 5, 9, 6, 7, 9, 8, 10, 5, 2, 1, 2, 3, 9, 3, 7, 5, 6, 6, 1, 1, 2, 9, 3, 4, 4, 5, 1, 3, 1, 4, 1, 3, 1, 1, 4, 4, 4, 2, 4, 1, 9, 9, 8, 5, 5, 6, 7, 8, 5, 8, 8, 2, 10, 4, 8, 8, 1, 10, 9, 4, 10, 5, 3, 3, 9, 7, 1, 2, 8, 6, 3, 3, 6, 10, 1, 2, 10, 7, 9, 6, 4, 5, 1, 2, 3, 1, 1, 7, 9, 9, 2, 6, 10, 8, 2, 1, 5, 6, 10, 8, 3, 1, 2, 4, 8, 2, 9, 8, 8, 6, 8, 1, 7, 8, 1, 5, 10, 1, 9, 6, 2, 1, 5, 1, 5, 4, 9, 2, 3, 4, 2, 3, 5, 3, 2, 10, 1, 3, 4, 7, 3, 4, 9, 3, 5, 2, 1, 2, 4, 1, 6, 6, 4, 2, 9, 8, 6, 9, 6, 5, 2, 6, 5, 2, 7, 5, 7, 8, 5, 4, 4, 4, 7, 9, 8, 2, 7, 8, 3, 5, 7, 4, 5, 2, 2, 4, 4, 7, 6, 5, 6, 10, 9, 5, 2, 6, 8, 10, 4, 1, 9, 7, 1, 2, 6, 7, 3, 7, 7, 4, 4, 10, 9, 2, 6, 4, 7, 8, 2, 6, 7, 1, 10, 5, 2, 2, 8, 3, 2, 8, 2, 2, 10, 10, 5, 7, 7, 2, 

3a) Trial of ACO with p = 100 and e = 0.9

In [11]:
ACO_BPP(100, 0.9, 1)

First generation: 
Fitness:    Best:  351   Worst:  568   Average:  458.81
    Best path:  [10, 1, 1, 6, 5, 7, 3, 4, 3, 2, 10, 10, 4, 5, 6, 10, 1, 10, 4, 8, 2, 4, 9, 6, 7, 6, 3, 3, 8, 2, 5, 4, 3, 1, 7, 9, 7, 3, 3, 1, 1, 7, 10, 7, 7, 2, 2, 9, 5, 1, 2, 5, 7, 4, 4, 5, 1, 4, 4, 10, 8, 5, 9, 2, 10, 5, 10, 1, 3, 9, 1, 4, 2, 5, 10, 6, 5, 3, 10, 4, 7, 4, 3, 2, 1, 4, 1, 8, 10, 5, 4, 6, 10, 8, 1, 3, 1, 4, 2, 1, 6, 5, 4, 8, 1, 3, 2, 6, 5, 8, 2, 10, 8, 2, 4, 5, 5, 2, 6, 10, 8, 2, 1, 5, 2, 8, 3, 7, 9, 3, 7, 1, 2, 6, 7, 6, 8, 5, 8, 4, 8, 8, 9, 3, 1, 7, 7, 8, 6, 4, 4, 1, 10, 5, 7, 8, 1, 4, 10, 7, 9, 4, 7, 6, 6, 1, 10, 4, 7, 1, 8, 6, 6, 5, 10, 1, 2, 9, 8, 5, 10, 5, 7, 2, 5, 9, 7, 6, 5, 7, 10, 1, 2, 5, 3, 3, 4, 5, 9, 6, 6, 9, 6, 4, 2, 9, 8, 10, 2, 7, 8, 5, 1, 2, 10, 5, 2, 6, 7, 4, 5, 3, 7, 9, 2, 3, 3, 10, 6, 1, 6, 4, 2, 2, 9, 9, 2, 4, 9, 7, 10, 1, 7, 5, 8, 3, 8, 8, 9, 5, 4, 8, 7, 3, 4, 9, 8, 1, 4, 1, 1, 3, 2, 1, 10, 2, 2, 3, 5, 2, 6, 6, 10, 1, 6, 8, 2, 3, 9, 1, 1, 5, 6, 7, 10, 4, 4, 4, 6, 7, 3, 6, 7, 2

3b) Trial of the ACO with p = 100 and e = 0.65

In [12]:
ACO_BPP(100, 0.65, 1)

First generation: 
Fitness:    Best:  377   Worst:  638   Average:  464.62
    Best path:  [2, 4, 2, 3, 7, 1, 9, 1, 9, 7, 5, 9, 6, 6, 1, 4, 4, 6, 9, 9, 2, 4, 6, 8, 8, 10, 1, 3, 4, 4, 8, 5, 5, 8, 3, 8, 9, 4, 4, 10, 9, 5, 6, 5, 3, 7, 5, 3, 9, 5, 7, 1, 5, 2, 7, 5, 7, 4, 3, 2, 4, 7, 8, 6, 4, 5, 7, 4, 6, 1, 2, 7, 9, 2, 10, 9, 8, 1, 1, 3, 9, 3, 4, 8, 4, 2, 10, 5, 10, 4, 10, 7, 8, 4, 4, 2, 10, 2, 3, 8, 8, 6, 5, 8, 7, 6, 6, 9, 9, 9, 1, 4, 2, 10, 4, 4, 10, 10, 3, 2, 1, 2, 6, 7, 8, 4, 7, 4, 9, 9, 10, 3, 10, 1, 4, 4, 10, 5, 6, 9, 5, 4, 3, 3, 2, 5, 5, 10, 6, 6, 9, 7, 5, 5, 4, 9, 4, 10, 5, 4, 7, 1, 8, 2, 5, 4, 6, 10, 8, 1, 1, 5, 3, 6, 2, 6, 4, 10, 6, 2, 2, 1, 5, 5, 5, 3, 9, 8, 6, 9, 7, 10, 4, 6, 5, 7, 2, 2, 6, 7, 2, 6, 4, 5, 2, 8, 3, 8, 6, 2, 9, 4, 8, 4, 6, 2, 1, 9, 10, 8, 10, 2, 10, 3, 10, 9, 9, 2, 7, 1, 4, 7, 1, 10, 3, 10, 2, 7, 5, 1, 8, 8, 4, 1, 3, 1, 9, 10, 7, 10, 8, 7, 4, 7, 1, 5, 2, 6, 9, 3, 8, 10, 1, 3, 4, 8, 3, 6, 5, 5, 8, 4, 2, 5, 2, 8, 8, 3, 5, 1, 1, 9, 8, 5, 4, 2, 4, 1, 1, 5, 6, 10, 9, 7

3c) Trial of ACO with p = 100 and e = 0.4

In [13]:
ACO_BPP(100, 0.4, 1)

First generation: 
Fitness:    Best:  353   Worst:  624   Average:  456.83
    Best path:  [3, 6, 7, 4, 10, 3, 5, 3, 3, 4, 10, 2, 4, 4, 9, 2, 7, 1, 2, 10, 3, 1, 8, 4, 4, 7, 8, 8, 7, 5, 5, 1, 3, 1, 5, 1, 2, 8, 1, 2, 1, 4, 1, 4, 3, 4, 9, 4, 2, 2, 8, 7, 6, 2, 1, 5, 9, 10, 3, 5, 1, 8, 4, 7, 2, 5, 4, 1, 7, 3, 3, 5, 7, 7, 1, 1, 8, 6, 5, 4, 10, 3, 2, 4, 7, 5, 7, 8, 9, 7, 10, 1, 8, 8, 1, 10, 8, 8, 5, 7, 1, 3, 3, 7, 4, 3, 3, 8, 4, 7, 9, 6, 2, 10, 7, 5, 6, 5, 8, 4, 1, 8, 3, 2, 3, 6, 2, 9, 2, 3, 7, 9, 6, 8, 2, 9, 10, 3, 2, 10, 5, 8, 1, 6, 3, 1, 3, 2, 2, 9, 7, 9, 9, 5, 9, 6, 3, 3, 7, 10, 8, 2, 2, 10, 7, 5, 6, 10, 3, 1, 4, 5, 3, 2, 8, 4, 1, 7, 7, 4, 5, 10, 5, 2, 7, 3, 1, 8, 10, 5, 10, 2, 4, 4, 4, 3, 3, 2, 1, 9, 7, 7, 1, 3, 5, 2, 5, 3, 5, 8, 6, 1, 2, 1, 6, 5, 1, 6, 4, 9, 4, 1, 8, 6, 9, 1, 2, 3, 5, 9, 1, 5, 7, 5, 10, 7, 9, 3, 3, 10, 5, 2, 2, 9, 5, 6, 7, 4, 3, 1, 7, 9, 7, 7, 4, 10, 5, 9, 2, 10, 9, 10, 10, 4, 1, 5, 7, 9, 5, 1, 7, 7, 2, 1, 4, 7, 5, 5, 7, 2, 4, 3, 3, 4, 9, 4, 9, 1, 1, 2, 9, 10, 4, 5, 6, 

3d) Trial of ACO with p = 100 and e = 0.15

In [14]:
ACO_BPP(100, 0.15, 1)

First generation: 
Fitness:    Best:  363   Worst:  573   Average:  462.85
    Best path:  [8, 10, 1, 3, 8, 5, 1, 9, 5, 6, 8, 6, 1, 4, 5, 6, 3, 3, 5, 8, 4, 1, 4, 6, 5, 2, 2, 8, 1, 8, 3, 5, 7, 10, 8, 7, 4, 5, 5, 7, 9, 6, 4, 4, 4, 6, 8, 5, 7, 9, 2, 2, 8, 6, 6, 7, 8, 9, 1, 2, 8, 2, 9, 6, 4, 2, 6, 2, 6, 5, 2, 7, 9, 4, 8, 4, 9, 5, 8, 9, 5, 1, 6, 6, 9, 8, 6, 1, 5, 10, 8, 7, 3, 8, 1, 3, 6, 9, 6, 7, 6, 8, 6, 3, 6, 7, 8, 5, 3, 6, 8, 5, 1, 8, 7, 2, 3, 8, 4, 7, 5, 6, 5, 8, 9, 9, 6, 10, 1, 5, 4, 8, 10, 4, 1, 5, 6, 8, 5, 1, 2, 2, 10, 10, 1, 2, 2, 8, 7, 2, 5, 7, 10, 6, 8, 10, 8, 4, 2, 9, 8, 5, 7, 3, 5, 3, 5, 5, 4, 5, 4, 9, 5, 9, 4, 10, 4, 10, 9, 10, 6, 4, 9, 9, 4, 9, 2, 3, 3, 1, 10, 1, 6, 10, 7, 7, 7, 10, 3, 2, 1, 10, 4, 7, 4, 5, 8, 10, 5, 4, 2, 2, 1, 3, 6, 6, 4, 7, 9, 1, 4, 6, 5, 5, 9, 1, 5, 9, 6, 10, 1, 7, 5, 8, 2, 2, 9, 9, 4, 4, 3, 9, 5, 5, 7, 8, 10, 5, 8, 9, 3, 3, 10, 5, 1, 6, 7, 5, 10, 8, 2, 3, 6, 6, 9, 9, 2, 3, 7, 1, 4, 4, 6, 6, 2, 9, 9, 6, 5, 4, 10, 8, 6, 9, 1, 1, 1, 7, 4, 1, 7, 7, 4, 3, 7, 8

4a) Trial of ACO with p = 50 and e = 0.9

In [15]:
ACO_BPP(50, 0.9, 1)

First generation: 
Fitness:    Best:  356   Worst:  622   Average:  474.72
    Best path:  [7, 2, 2, 6, 2, 7, 3, 3, 8, 6, 6, 10, 4, 8, 7, 1, 9, 1, 3, 7, 9, 8, 8, 9, 5, 5, 1, 7, 1, 6, 1, 3, 6, 3, 8, 7, 5, 3, 8, 7, 8, 8, 5, 3, 6, 1, 5, 3, 10, 2, 1, 10, 4, 3, 5, 3, 4, 5, 5, 9, 9, 3, 8, 3, 8, 5, 5, 6, 6, 6, 6, 5, 4, 5, 7, 6, 1, 6, 10, 2, 1, 2, 2, 6, 9, 5, 2, 4, 7, 7, 5, 1, 5, 6, 6, 7, 6, 8, 5, 7, 4, 1, 9, 10, 6, 5, 2, 6, 8, 2, 6, 8, 7, 8, 7, 3, 5, 1, 10, 3, 7, 1, 7, 9, 9, 2, 4, 4, 5, 5, 7, 5, 10, 4, 6, 6, 9, 9, 7, 7, 10, 6, 7, 3, 2, 4, 4, 7, 6, 8, 4, 9, 2, 3, 7, 8, 10, 10, 8, 1, 10, 4, 1, 7, 7, 10, 1, 4, 6, 9, 6, 1, 9, 9, 3, 5, 5, 9, 1, 4, 10, 4, 9, 2, 6, 6, 6, 8, 4, 10, 2, 1, 3, 3, 1, 9, 4, 4, 5, 1, 3, 8, 2, 1, 1, 7, 6, 10, 2, 7, 1, 1, 1, 3, 7, 7, 2, 1, 6, 1, 9, 8, 7, 3, 4, 2, 4, 9, 5, 5, 3, 2, 2, 4, 3, 8, 7, 4, 7, 2, 6, 6, 1, 5, 4, 1, 1, 4, 1, 8, 5, 2, 7, 7, 10, 6, 9, 2, 10, 8, 7, 6, 4, 2, 4, 2, 4, 8, 3, 7, 1, 2, 4, 1, 5, 7, 5, 2, 7, 1, 6, 10, 4, 7, 3, 9, 9, 10, 2, 4, 7, 4, 1, 1, 9, 5, 3

4b) Trial of the ACO with p = 50 and e = 0.65

In [16]:
ACO_BPP(50, 0.65, 1)

First generation: 
Fitness:    Best:  361   Worst:  580   Average:  470.02
    Best path:  [2, 1, 10, 4, 6, 2, 8, 6, 2, 9, 5, 8, 3, 6, 1, 5, 4, 2, 2, 2, 4, 6, 6, 7, 9, 1, 4, 8, 1, 7, 1, 3, 7, 10, 2, 1, 3, 9, 1, 2, 9, 7, 10, 3, 5, 5, 4, 6, 9, 1, 4, 2, 10, 9, 1, 2, 10, 8, 7, 4, 1, 6, 5, 6, 7, 6, 8, 5, 6, 7, 7, 2, 7, 8, 8, 10, 4, 4, 9, 6, 7, 2, 8, 10, 2, 10, 8, 4, 1, 8, 8, 4, 10, 3, 4, 10, 3, 4, 10, 10, 2, 4, 5, 7, 1, 10, 6, 1, 1, 5, 2, 2, 3, 4, 5, 8, 3, 1, 9, 8, 9, 8, 9, 3, 10, 4, 4, 10, 2, 8, 10, 4, 1, 7, 4, 3, 9, 9, 6, 2, 1, 3, 5, 8, 10, 1, 1, 3, 5, 6, 4, 2, 8, 5, 7, 6, 9, 5, 4, 10, 8, 6, 8, 4, 4, 1, 3, 1, 2, 1, 5, 10, 8, 3, 6, 4, 1, 8, 9, 7, 5, 5, 2, 1, 9, 5, 8, 8, 8, 4, 7, 7, 7, 10, 3, 2, 4, 3, 7, 1, 6, 4, 1, 3, 8, 3, 2, 6, 5, 1, 8, 6, 6, 2, 3, 4, 9, 7, 5, 6, 9, 5, 4, 2, 8, 7, 5, 9, 3, 7, 3, 9, 10, 7, 1, 10, 10, 5, 10, 2, 8, 4, 10, 4, 9, 6, 5, 6, 6, 6, 10, 9, 2, 3, 6, 10, 9, 2, 3, 7, 4, 3, 8, 5, 6, 5, 1, 9, 3, 5, 3, 3, 4, 5, 8, 3, 2, 10, 3, 8, 5, 7, 10, 2, 8, 2, 6, 2, 6, 8, 9, 4, 4, 

4c) Trial of ACO with p = 50 and e = 0.4

In [17]:
ACO_BPP(50, 0.4, 1)

First generation: 
Fitness:    Best:  384   Worst:  609   Average:  477.16
    Best path:  [5, 3, 5, 9, 9, 3, 1, 3, 9, 4, 2, 3, 5, 1, 6, 6, 10, 7, 5, 1, 1, 2, 3, 5, 2, 4, 2, 10, 6, 5, 4, 6, 3, 7, 2, 5, 8, 1, 8, 7, 10, 6, 8, 7, 5, 4, 10, 3, 2, 1, 5, 5, 5, 10, 8, 6, 9, 9, 4, 5, 3, 7, 2, 10, 6, 8, 2, 1, 1, 5, 4, 7, 4, 1, 4, 10, 7, 4, 8, 2, 10, 5, 3, 10, 8, 6, 7, 8, 10, 7, 4, 2, 1, 1, 4, 4, 1, 9, 7, 3, 9, 9, 1, 7, 1, 5, 9, 9, 9, 6, 10, 4, 10, 9, 3, 5, 5, 7, 6, 6, 5, 9, 6, 9, 5, 1, 3, 1, 2, 4, 4, 7, 9, 5, 6, 8, 3, 9, 5, 8, 3, 10, 5, 6, 10, 1, 5, 7, 2, 3, 7, 5, 3, 6, 8, 4, 4, 1, 9, 10, 9, 10, 3, 7, 5, 9, 1, 1, 5, 3, 8, 8, 6, 5, 10, 9, 2, 3, 6, 6, 4, 9, 2, 5, 2, 5, 7, 5, 7, 6, 7, 7, 9, 2, 1, 6, 6, 4, 5, 4, 4, 2, 4, 7, 8, 7, 3, 4, 7, 9, 7, 2, 8, 4, 8, 8, 4, 7, 4, 8, 5, 4, 7, 6, 7, 1, 5, 2, 7, 5, 5, 3, 4, 5, 1, 9, 4, 2, 1, 8, 10, 5, 4, 1, 9, 8, 9, 3, 3, 2, 10, 5, 4, 6, 9, 8, 9, 3, 4, 3, 10, 2, 3, 6, 4, 2, 5, 6, 1, 3, 3, 8, 4, 4, 2, 10, 1, 4, 4, 2, 9, 5, 2, 1, 2, 10, 6, 4, 3, 2, 6, 7, 4, 5, 8, 5

4d) Trial of ACO with p = 50 and e = 0.15

In [18]:
ACO_BPP(50, 0.15, 1)

First generation: 
Fitness:    Best:  375   Worst:  589   Average:  477.78
    Best path:  [4, 5, 4, 1, 10, 9, 3, 4, 5, 10, 6, 3, 3, 2, 5, 3, 6, 1, 6, 6, 5, 5, 2, 10, 3, 3, 3, 6, 7, 4, 4, 8, 4, 1, 6, 1, 6, 4, 6, 4, 3, 3, 3, 8, 5, 1, 3, 6, 1, 6, 8, 4, 2, 8, 7, 9, 3, 1, 1, 1, 4, 2, 9, 5, 4, 6, 7, 4, 10, 7, 3, 2, 2, 1, 6, 7, 10, 2, 2, 2, 6, 4, 3, 3, 8, 8, 6, 1, 4, 9, 10, 4, 7, 4, 1, 6, 4, 2, 3, 2, 7, 9, 8, 3, 3, 8, 8, 2, 3, 6, 7, 2, 8, 9, 8, 3, 10, 2, 7, 1, 2, 9, 6, 4, 7, 6, 9, 10, 10, 1, 9, 2, 7, 2, 10, 10, 1, 9, 5, 10, 7, 4, 3, 7, 5, 9, 4, 7, 4, 6, 9, 3, 8, 4, 10, 8, 10, 2, 7, 5, 2, 2, 3, 4, 2, 2, 7, 4, 1, 3, 10, 1, 4, 5, 2, 7, 3, 2, 4, 6, 5, 7, 5, 8, 3, 7, 6, 3, 10, 9, 10, 4, 8, 8, 5, 4, 5, 6, 1, 8, 8, 4, 8, 9, 3, 5, 3, 4, 7, 3, 3, 9, 10, 5, 4, 8, 10, 9, 5, 3, 1, 7, 1, 2, 8, 1, 8, 2, 10, 4, 4, 2, 10, 3, 2, 4, 3, 3, 6, 8, 9, 3, 8, 5, 10, 6, 1, 6, 3, 5, 8, 9, 1, 8, 10, 7, 6, 7, 10, 1, 3, 2, 8, 10, 2, 3, 9, 6, 6, 2, 9, 8, 7, 4, 3, 3, 9, 8, 2, 1, 4, 8, 5, 9, 1, 4, 2, 3, 5, 9, 1, 3, 7, 3, 1

# Problem 2

For problem 2, there are 50 bins, 500 items. The weight of items are 1/2,4/2,9/2,...,(500^2)/2. 

The algorithm aims at utilizing p number of ants and 50000/p iterations to find an optimal ant path for putting items of different weights into a certain number of bins such that the maximum weight difference of heaviest and lightest bin is kept to a minimum.

The algorithm prints the fitness 'Best', 'Worst' & 'Avereage'of the first and last generation, and also the best path and worst path of first and last generation. Note: a small value in fitness indicates a better path.


1a) Trial of ACO with p = 200 and e = 0.9

In [19]:
ACO_BPP(200, 0.9, 2)

First generation: 
Fitness:    Best:  11131.0   Worst:  27607.0   Average:  16194.4025
    Best path:  [37, 32, 19, 12, 14, 2, 15, 48, 13, 23, 46, 30, 5, 45, 38, 8, 44, 4, 25, 41, 1, 5, 18, 11, 10, 44, 14, 31, 23, 39, 40, 1, 13, 13, 38, 9, 9, 9, 24, 27, 19, 36, 19, 11, 9, 43, 15, 22, 36, 12, 22, 49, 16, 18, 13, 31, 27, 11, 27, 20, 45, 10, 3, 2, 4, 31, 32, 6, 14, 37, 27, 43, 23, 10, 9, 13, 15, 5, 30, 49, 19, 19, 5, 31, 30, 50, 38, 21, 36, 23, 28, 33, 11, 26, 43, 6, 34, 10, 40, 5, 14, 39, 48, 31, 18, 21, 30, 16, 8, 5, 5, 5, 15, 15, 14, 23, 8, 7, 6, 32, 18, 42, 11, 45, 21, 15, 4, 9, 40, 24, 14, 5, 35, 27, 34, 19, 35, 46, 25, 3, 22, 8, 35, 41, 7, 1, 2, 12, 25, 40, 6, 2, 45, 9, 10, 46, 17, 20, 41, 37, 7, 19, 7, 46, 8, 31, 41, 13, 26, 20, 15, 25, 50, 7, 25, 15, 49, 5, 22, 24, 22, 31, 34, 8, 26, 14, 8, 28, 13, 21, 37, 3, 30, 45, 19, 49, 5, 43, 42, 34, 16, 23, 15, 25, 26, 48, 48, 45, 44, 16, 15, 8, 45, 50, 30, 16, 17, 37, 46, 6, 19, 35, 6, 14, 32, 2, 7, 12, 18, 41, 5, 48, 13, 23, 42, 13, 16, 2

1b) Trial of the ACO with p = 200 and e = 0.65

In [20]:
ACO_BPP(200, 0.65, 2)

First generation: 
Fitness:    Best:  11010.5   Worst:  27609.0   Average:  16284.0975
    Best path:  [49, 20, 19, 9, 7, 41, 44, 33, 16, 9, 26, 8, 15, 33, 48, 41, 49, 8, 17, 4, 42, 38, 27, 25, 45, 33, 11, 12, 35, 10, 11, 19, 1, 7, 8, 32, 36, 21, 29, 9, 7, 41, 3, 30, 18, 25, 42, 3, 40, 28, 7, 20, 49, 39, 48, 6, 32, 49, 12, 30, 8, 43, 6, 34, 42, 11, 30, 28, 18, 7, 18, 27, 40, 37, 3, 21, 18, 1, 50, 31, 6, 8, 47, 17, 45, 17, 4, 42, 12, 19, 25, 36, 40, 8, 43, 36, 3, 31, 46, 46, 25, 24, 22, 35, 32, 28, 46, 49, 8, 32, 3, 20, 3, 24, 38, 8, 34, 29, 21, 26, 13, 1, 2, 25, 35, 26, 18, 9, 33, 38, 25, 37, 21, 14, 18, 23, 34, 15, 3, 2, 41, 38, 20, 31, 20, 6, 14, 36, 6, 13, 10, 41, 21, 23, 15, 12, 18, 31, 44, 37, 14, 4, 44, 17, 48, 35, 11, 30, 27, 21, 34, 2, 25, 16, 18, 39, 39, 17, 25, 37, 32, 36, 29, 18, 28, 33, 5, 14, 20, 27, 15, 5, 17, 13, 35, 47, 3, 27, 46, 40, 29, 20, 44, 26, 14, 50, 24, 17, 50, 24, 24, 12, 30, 29, 29, 45, 49, 1, 36, 20, 29, 15, 38, 17, 36, 28, 44, 7, 13, 1, 7, 40, 15, 43, 27, 4

1c) Trial of ACO with p = 200 and e = 0.4

In [21]:
ACO_BPP(200, 0.4, 2)

First generation: 
Fitness:    Best:  11512.5   Worst:  26405.0   Average:  16206.07
    Best path:  [34, 35, 5, 8, 26, 40, 38, 4, 12, 12, 40, 4, 3, 41, 35, 48, 46, 10, 39, 22, 40, 47, 34, 42, 45, 13, 15, 28, 50, 20, 33, 27, 24, 9, 10, 33, 41, 37, 9, 11, 47, 47, 46, 10, 2, 44, 1, 5, 30, 33, 1, 49, 43, 20, 9, 48, 37, 16, 19, 5, 46, 10, 45, 27, 38, 17, 44, 26, 27, 1, 17, 43, 34, 31, 48, 2, 47, 3, 4, 16, 29, 18, 26, 24, 37, 32, 8, 39, 3, 40, 35, 29, 19, 1, 10, 47, 31, 33, 6, 26, 28, 42, 16, 44, 31, 27, 38, 36, 3, 5, 50, 31, 36, 37, 29, 7, 19, 36, 28, 8, 39, 17, 36, 38, 6, 8, 45, 37, 30, 45, 30, 18, 28, 18, 12, 22, 25, 24, 35, 23, 36, 24, 41, 4, 7, 32, 28, 21, 28, 44, 47, 5, 5, 27, 27, 16, 43, 4, 31, 1, 32, 1, 7, 28, 6, 32, 46, 8, 10, 50, 45, 18, 21, 30, 42, 27, 16, 32, 2, 31, 38, 4, 30, 48, 36, 26, 34, 29, 29, 19, 8, 15, 50, 12, 36, 35, 11, 37, 43, 35, 7, 26, 34, 46, 3, 24, 49, 14, 47, 20, 8, 22, 20, 19, 9, 50, 24, 50, 32, 42, 14, 9, 12, 3, 36, 4, 34, 32, 12, 17, 26, 23, 35, 30, 10, 50, 3

1d) Trial of ACO with p = 200 and e = 0.15

In [22]:
ACO_BPP(200, 0.15, 2)

First generation: 
Fitness:    Best:  11039.0   Worst:  27493.5   Average:  16510.52
    Best path:  [1, 49, 26, 24, 29, 8, 45, 5, 4, 34, 3, 4, 1, 41, 36, 31, 16, 11, 5, 28, 47, 7, 11, 35, 20, 9, 26, 34, 34, 43, 39, 8, 40, 8, 9, 47, 8, 14, 21, 45, 42, 16, 27, 1, 27, 33, 15, 13, 29, 1, 14, 45, 11, 40, 13, 28, 42, 37, 17, 34, 2, 26, 39, 15, 1, 23, 1, 44, 2, 20, 7, 47, 10, 17, 50, 30, 28, 17, 12, 28, 7, 39, 43, 11, 3, 23, 41, 3, 36, 4, 15, 27, 36, 23, 30, 33, 18, 19, 40, 34, 42, 34, 38, 41, 1, 50, 15, 30, 4, 14, 41, 2, 33, 11, 21, 42, 3, 1, 16, 2, 14, 4, 17, 38, 49, 33, 41, 37, 17, 34, 44, 43, 14, 15, 44, 12, 48, 26, 44, 32, 10, 11, 14, 16, 18, 36, 48, 28, 8, 12, 17, 19, 3, 17, 47, 13, 37, 17, 22, 48, 31, 50, 2, 33, 5, 17, 5, 41, 38, 49, 38, 26, 3, 14, 19, 8, 11, 49, 21, 43, 47, 45, 31, 42, 22, 5, 9, 30, 21, 29, 44, 45, 23, 4, 2, 33, 36, 31, 29, 21, 29, 33, 44, 8, 22, 19, 47, 34, 29, 21, 27, 26, 31, 3, 42, 24, 11, 18, 3, 38, 26, 36, 40, 6, 39, 26, 25, 24, 43, 35, 27, 50, 38, 15, 48, 25, 4

2a) Trial of ACO with p = 150 and e = 0.9

In [23]:
ACO_BPP(150, 0.9, 2)

First generation: 
Fitness:    Best:  12014.0   Worst:  26501.5   Average:  16748.95
    Best path:  [30, 9, 17, 20, 32, 24, 36, 44, 39, 30, 36, 34, 22, 38, 26, 43, 23, 38, 7, 35, 38, 26, 38, 34, 6, 26, 37, 18, 7, 3, 35, 24, 27, 21, 49, 45, 17, 7, 4, 26, 32, 29, 4, 41, 29, 46, 29, 48, 49, 39, 35, 43, 3, 14, 13, 12, 3, 2, 30, 13, 6, 13, 31, 8, 1, 13, 41, 40, 49, 34, 12, 5, 20, 1, 38, 11, 20, 4, 39, 41, 12, 36, 5, 49, 19, 23, 8, 14, 40, 32, 31, 21, 4, 36, 49, 8, 9, 19, 42, 27, 3, 49, 12, 23, 30, 21, 38, 46, 33, 4, 35, 41, 48, 28, 25, 14, 39, 37, 28, 29, 4, 40, 34, 35, 27, 45, 47, 1, 28, 43, 14, 21, 16, 7, 12, 33, 3, 46, 14, 47, 35, 44, 40, 10, 48, 16, 28, 42, 9, 8, 42, 42, 44, 44, 16, 45, 31, 11, 40, 31, 12, 48, 12, 35, 18, 10, 6, 26, 15, 21, 26, 21, 37, 13, 27, 13, 20, 47, 2, 35, 38, 5, 28, 44, 1, 38, 46, 5, 13, 46, 7, 27, 41, 1, 2, 14, 18, 20, 13, 31, 24, 27, 31, 25, 6, 12, 31, 28, 5, 15, 6, 7, 24, 9, 7, 10, 34, 13, 35, 27, 22, 50, 28, 42, 19, 7, 32, 3, 43, 37, 40, 5, 39, 45, 49, 11, 4

2b) Trial of the ACO with p = 150 and e = 0.65

In [24]:
ACO_BPP(150, 0.65, 2)

First generation: 
Fitness:    Best:  11133.0   Worst:  28796.5   Average:  16290.176666666666
    Best path:  [19, 27, 28, 2, 21, 20, 19, 30, 49, 15, 49, 31, 28, 35, 14, 34, 11, 17, 2, 41, 17, 42, 50, 34, 30, 30, 3, 6, 30, 2, 25, 39, 5, 11, 23, 40, 40, 35, 50, 2, 26, 9, 13, 17, 43, 31, 16, 26, 16, 2, 37, 29, 29, 7, 6, 27, 22, 20, 2, 36, 38, 23, 22, 26, 22, 42, 3, 7, 45, 22, 28, 33, 38, 27, 35, 45, 16, 22, 11, 10, 45, 18, 5, 6, 42, 10, 11, 3, 16, 41, 32, 6, 18, 33, 22, 14, 40, 17, 10, 44, 33, 30, 34, 47, 13, 29, 47, 14, 38, 27, 10, 4, 25, 30, 2, 4, 17, 6, 21, 19, 10, 19, 24, 9, 37, 25, 15, 24, 6, 31, 30, 26, 24, 25, 15, 6, 3, 50, 7, 28, 16, 15, 33, 33, 39, 43, 26, 5, 39, 11, 4, 13, 43, 48, 25, 12, 10, 18, 27, 20, 4, 33, 33, 28, 5, 49, 24, 7, 38, 11, 13, 1, 45, 41, 41, 25, 36, 10, 2, 31, 7, 5, 16, 17, 7, 10, 31, 46, 6, 18, 9, 42, 26, 8, 42, 34, 31, 9, 17, 50, 44, 21, 45, 38, 42, 50, 23, 42, 24, 24, 47, 12, 16, 50, 22, 34, 28, 22, 42, 6, 35, 32, 45, 37, 34, 38, 30, 34, 27, 47, 31, 27, 34

2c) Trial of ACO with p = 150 and e = 0.4

In [25]:
ACO_BPP(150, 0.4, 2)

First generation: 
Fitness:    Best:  11245.5   Worst:  24006.5   Average:  16408.853333333333
    Best path:  [17, 36, 45, 18, 4, 23, 45, 41, 37, 7, 16, 29, 20, 38, 18, 30, 7, 3, 9, 8, 39, 32, 26, 1, 31, 23, 14, 9, 36, 17, 34, 15, 21, 4, 15, 5, 26, 1, 2, 5, 3, 17, 17, 30, 6, 30, 25, 18, 11, 32, 20, 11, 30, 20, 16, 13, 38, 34, 46, 36, 27, 50, 10, 30, 18, 14, 18, 11, 14, 44, 42, 17, 4, 34, 23, 45, 3, 34, 31, 19, 5, 47, 32, 31, 6, 15, 5, 34, 4, 43, 10, 35, 38, 17, 20, 33, 11, 11, 17, 4, 20, 45, 41, 38, 44, 3, 39, 49, 20, 5, 31, 31, 40, 3, 17, 30, 29, 43, 31, 49, 31, 13, 21, 40, 23, 27, 2, 8, 21, 26, 31, 15, 12, 14, 40, 24, 27, 3, 5, 18, 18, 14, 8, 30, 13, 13, 28, 48, 4, 9, 35, 46, 31, 26, 5, 23, 34, 48, 16, 37, 38, 21, 2, 17, 4, 28, 23, 30, 19, 17, 18, 25, 29, 42, 32, 23, 36, 10, 9, 17, 23, 29, 2, 17, 8, 16, 35, 32, 45, 31, 44, 19, 29, 1, 49, 29, 17, 22, 32, 25, 16, 47, 26, 5, 41, 7, 49, 32, 38, 17, 41, 41, 2, 20, 19, 35, 29, 26, 2, 19, 11, 30, 38, 37, 9, 35, 24, 45, 12, 42, 42, 26, 39, 

2d) Trial of ACO with p = 150 and e = 0.15

In [26]:
ACO_BPP(150, 0.15, 2)

First generation: 
Fitness:    Best:  11610.0   Worst:  24293.0   Average:  16347.903333333334
    Best path:  [6, 27, 16, 8, 44, 28, 2, 25, 29, 37, 29, 39, 5, 21, 37, 7, 41, 13, 8, 32, 34, 13, 21, 32, 11, 17, 2, 3, 16, 3, 11, 27, 41, 34, 15, 36, 42, 10, 26, 38, 10, 1, 48, 15, 17, 39, 22, 1, 11, 48, 43, 37, 43, 46, 33, 17, 32, 35, 48, 4, 2, 39, 39, 26, 14, 6, 13, 33, 29, 21, 1, 7, 16, 7, 22, 43, 49, 50, 8, 25, 33, 14, 39, 16, 1, 17, 49, 39, 16, 43, 17, 35, 12, 18, 7, 7, 34, 9, 10, 4, 36, 7, 11, 39, 16, 15, 17, 25, 6, 44, 20, 35, 16, 25, 34, 47, 40, 14, 18, 11, 32, 17, 20, 4, 25, 1, 41, 1, 3, 22, 8, 24, 3, 6, 28, 32, 36, 40, 36, 36, 29, 45, 16, 4, 10, 2, 13, 20, 35, 28, 30, 47, 20, 8, 44, 48, 8, 40, 45, 22, 41, 14, 37, 32, 8, 48, 14, 8, 36, 19, 10, 3, 9, 43, 37, 27, 33, 20, 29, 46, 24, 46, 4, 47, 29, 29, 48, 25, 26, 2, 50, 16, 23, 7, 26, 11, 50, 30, 27, 50, 28, 18, 49, 5, 5, 12, 17, 40, 18, 45, 38, 8, 18, 19, 18, 28, 13, 38, 24, 24, 23, 21, 6, 28, 2, 1, 25, 31, 19, 7, 2, 25, 30, 2, 46, 

3a) Trial of ACO with p = 100 and e = 0.9

In [27]:
ACO_BPP(100, 0.9, 2)

First generation: 
Fitness:    Best:  12162.5   Worst:  27605.5   Average:  16585.905
    Best path:  [42, 37, 7, 33, 24, 3, 41, 26, 39, 36, 24, 18, 16, 14, 49, 16, 9, 7, 25, 37, 45, 20, 50, 33, 24, 28, 2, 49, 44, 32, 12, 30, 4, 39, 18, 38, 15, 14, 2, 16, 2, 3, 12, 42, 35, 5, 28, 13, 44, 15, 29, 43, 21, 12, 36, 19, 36, 38, 38, 19, 39, 43, 15, 18, 10, 14, 15, 48, 16, 7, 32, 28, 42, 46, 10, 20, 16, 21, 27, 18, 21, 15, 12, 14, 39, 39, 8, 20, 39, 43, 36, 16, 29, 35, 32, 23, 28, 42, 50, 38, 19, 39, 30, 41, 5, 7, 7, 34, 26, 33, 19, 43, 41, 50, 16, 27, 40, 32, 36, 2, 13, 2, 18, 9, 22, 8, 39, 1, 1, 10, 26, 30, 33, 7, 14, 9, 42, 28, 12, 16, 48, 7, 38, 27, 5, 1, 22, 28, 18, 24, 16, 37, 28, 22, 8, 1, 20, 36, 43, 23, 17, 16, 31, 6, 18, 34, 33, 39, 17, 4, 10, 29, 35, 21, 25, 24, 43, 30, 12, 6, 17, 31, 29, 37, 40, 22, 25, 24, 6, 10, 28, 43, 25, 24, 11, 33, 6, 34, 35, 30, 36, 47, 30, 10, 23, 31, 18, 15, 10, 47, 25, 25, 10, 35, 11, 3, 38, 16, 7, 28, 16, 40, 43, 27, 33, 47, 12, 14, 33, 27, 1, 4, 33, 25

3b) Trial of the ACO with p = 100 and e = 0.65

In [28]:
ACO_BPP(100, 0.65, 2)

First generation: 
Fitness:    Best:  11631.0   Worst:  28804.0   Average:  16491.845
    Best path:  [44, 9, 32, 25, 45, 39, 17, 7, 48, 47, 43, 10, 12, 41, 21, 29, 30, 5, 13, 19, 37, 22, 30, 20, 36, 28, 45, 22, 10, 49, 38, 3, 39, 4, 28, 33, 4, 44, 49, 28, 35, 33, 7, 17, 3, 34, 31, 43, 11, 34, 32, 31, 16, 28, 44, 13, 10, 25, 29, 44, 31, 42, 6, 24, 7, 19, 9, 4, 7, 42, 27, 2, 12, 15, 50, 23, 36, 47, 13, 25, 26, 40, 13, 35, 6, 1, 3, 18, 25, 27, 25, 4, 27, 27, 10, 38, 38, 3, 30, 14, 42, 5, 17, 44, 28, 18, 9, 48, 44, 12, 40, 12, 7, 16, 34, 46, 23, 4, 39, 48, 20, 13, 19, 47, 45, 42, 8, 39, 4, 42, 20, 25, 15, 24, 32, 7, 23, 48, 44, 2, 1, 12, 35, 29, 39, 2, 35, 27, 4, 11, 32, 42, 34, 25, 9, 19, 34, 2, 13, 32, 18, 15, 45, 17, 38, 26, 30, 31, 25, 45, 5, 36, 39, 49, 35, 41, 32, 2, 35, 45, 25, 33, 11, 19, 43, 25, 48, 28, 6, 5, 13, 30, 10, 38, 43, 44, 50, 31, 22, 33, 23, 26, 27, 11, 36, 41, 22, 7, 35, 45, 15, 37, 16, 14, 46, 39, 11, 1, 47, 23, 31, 9, 38, 11, 5, 38, 37, 33, 11, 18, 46, 46, 38, 2, 16

3c) Trial of ACO with p = 100 and e = 0.4

In [29]:
ACO_BPP(100, 0.4, 2)

First generation: 
Fitness:    Best:  11458.0   Worst:  25338.0   Average:  16655.99
    Best path:  [2, 6, 9, 18, 33, 40, 15, 19, 6, 2, 47, 17, 8, 8, 37, 24, 21, 23, 8, 46, 14, 42, 7, 3, 35, 33, 37, 16, 37, 35, 29, 45, 17, 8, 26, 42, 34, 37, 36, 45, 27, 42, 37, 24, 16, 9, 45, 17, 4, 3, 35, 48, 7, 3, 43, 40, 11, 49, 6, 38, 32, 43, 8, 12, 36, 50, 36, 46, 27, 39, 39, 22, 26, 15, 50, 23, 6, 37, 6, 8, 20, 30, 13, 38, 30, 29, 10, 14, 19, 21, 44, 19, 25, 44, 31, 36, 13, 31, 2, 7, 24, 16, 42, 12, 12, 2, 26, 43, 40, 2, 6, 1, 13, 9, 13, 22, 3, 16, 36, 42, 31, 47, 33, 36, 34, 27, 42, 4, 26, 28, 39, 3, 8, 16, 32, 23, 36, 21, 5, 27, 17, 11, 6, 15, 15, 5, 2, 7, 36, 43, 30, 9, 14, 20, 39, 41, 16, 37, 5, 5, 33, 46, 45, 39, 30, 48, 14, 1, 34, 28, 34, 3, 4, 4, 33, 11, 29, 22, 46, 3, 2, 32, 31, 15, 23, 44, 50, 40, 13, 23, 30, 29, 45, 5, 27, 46, 1, 5, 6, 3, 2, 26, 1, 5, 42, 29, 37, 28, 19, 32, 38, 9, 14, 17, 23, 16, 39, 1, 5, 26, 20, 13, 10, 2, 36, 21, 1, 40, 36, 42, 6, 50, 13, 1, 17, 1, 20, 6, 30, 30, 7

3d) Trial of ACO with p = 100 and e = 0.15

In [30]:
ACO_BPP(100, 0.15, 2)

First generation: 
Fitness:    Best:  11514.0   Worst:  28808.5   Average:  16191.42
    Best path:  [17, 24, 38, 21, 21, 3, 14, 21, 33, 44, 38, 35, 29, 3, 12, 21, 22, 42, 16, 22, 50, 48, 6, 40, 10, 38, 44, 27, 41, 16, 35, 17, 38, 11, 6, 46, 30, 32, 10, 21, 5, 4, 15, 14, 9, 33, 7, 33, 7, 6, 34, 15, 36, 49, 42, 19, 19, 25, 48, 39, 46, 20, 40, 24, 22, 36, 41, 17, 37, 13, 40, 34, 11, 4, 17, 47, 14, 38, 14, 14, 34, 34, 17, 37, 4, 35, 10, 29, 21, 38, 5, 34, 33, 33, 50, 44, 25, 1, 1, 23, 14, 8, 6, 3, 36, 15, 8, 4, 13, 9, 33, 24, 1, 30, 8, 13, 9, 13, 33, 14, 38, 8, 30, 40, 2, 24, 22, 17, 25, 31, 14, 36, 10, 16, 3, 25, 33, 4, 32, 9, 22, 41, 41, 43, 19, 27, 9, 47, 41, 26, 34, 32, 14, 26, 25, 12, 14, 14, 42, 32, 40, 7, 22, 41, 45, 32, 30, 32, 4, 15, 13, 3, 32, 45, 29, 22, 42, 16, 43, 8, 49, 39, 18, 29, 6, 40, 50, 2, 34, 39, 9, 11, 15, 17, 5, 32, 24, 18, 46, 32, 30, 1, 15, 1, 47, 38, 40, 33, 6, 45, 49, 43, 43, 24, 6, 31, 36, 49, 6, 35, 3, 48, 34, 3, 22, 18, 5, 37, 17, 7, 49, 8, 16, 8, 17, 36, 50,

4a) Trial of ACO with p = 50 and e = 0.9

In [31]:
ACO_BPP(50, 0.9, 2)

First generation: 
Fitness:    Best:  11132.5   Worst:  23743.0   Average:  16420.24
    Best path:  [31, 23, 38, 40, 27, 24, 44, 15, 44, 37, 40, 13, 36, 10, 27, 28, 24, 47, 44, 11, 39, 20, 30, 6, 32, 28, 14, 50, 32, 19, 42, 18, 16, 34, 35, 11, 29, 5, 18, 35, 41, 18, 24, 23, 39, 20, 13, 10, 29, 36, 8, 28, 32, 46, 42, 7, 7, 46, 35, 4, 34, 47, 40, 30, 43, 23, 9, 1, 31, 34, 37, 36, 41, 35, 35, 3, 43, 41, 22, 40, 28, 26, 26, 17, 38, 27, 27, 4, 33, 35, 26, 15, 45, 45, 6, 23, 47, 25, 50, 22, 45, 17, 21, 19, 40, 16, 25, 41, 15, 12, 27, 48, 13, 43, 43, 15, 35, 46, 3, 13, 6, 22, 10, 45, 18, 36, 16, 10, 26, 39, 36, 33, 18, 10, 19, 45, 47, 45, 26, 49, 30, 42, 11, 23, 3, 22, 28, 45, 31, 21, 5, 16, 40, 25, 36, 46, 49, 22, 4, 33, 11, 16, 24, 21, 31, 45, 17, 35, 16, 22, 20, 6, 44, 34, 16, 24, 36, 25, 8, 15, 41, 10, 39, 10, 36, 3, 29, 3, 47, 27, 18, 3, 50, 37, 3, 40, 8, 26, 22, 4, 24, 17, 9, 37, 30, 1, 26, 4, 16, 43, 36, 2, 8, 4, 21, 19, 5, 49, 16, 47, 1, 14, 24, 34, 43, 29, 30, 16, 12, 40, 28, 42, 5,

4b) Trial of the ACO with p = 50 and e = 0.65

In [32]:
ACO_BPP(50, 0.65, 2)

First generation: 
Fitness:    Best:  11611.0   Worst:  22495.5   Average:  16766.39
    Best path:  [9, 35, 22, 3, 23, 37, 15, 38, 11, 27, 11, 31, 23, 41, 44, 6, 14, 50, 44, 40, 20, 16, 6, 1, 34, 49, 37, 10, 13, 28, 38, 10, 18, 32, 19, 30, 14, 33, 19, 10, 42, 23, 17, 7, 24, 11, 11, 36, 9, 21, 20, 14, 8, 4, 12, 27, 28, 21, 33, 20, 7, 20, 47, 20, 4, 36, 3, 48, 28, 4, 34, 38, 31, 35, 1, 13, 15, 40, 6, 7, 3, 6, 4, 33, 40, 3, 39, 2, 12, 37, 10, 20, 5, 17, 32, 49, 27, 39, 8, 23, 17, 2, 25, 28, 28, 28, 4, 14, 40, 35, 27, 28, 18, 2, 18, 28, 26, 7, 41, 11, 17, 25, 31, 9, 5, 26, 14, 7, 16, 43, 31, 14, 50, 6, 33, 18, 46, 8, 13, 16, 3, 20, 46, 31, 11, 42, 4, 12, 30, 18, 9, 46, 16, 40, 48, 45, 29, 3, 35, 16, 32, 15, 11, 36, 34, 6, 10, 23, 11, 18, 37, 24, 38, 43, 41, 46, 43, 39, 25, 3, 21, 33, 50, 19, 19, 1, 9, 40, 42, 3, 22, 17, 24, 4, 4, 19, 19, 40, 24, 14, 15, 39, 16, 15, 5, 17, 1, 32, 30, 20, 37, 18, 40, 22, 27, 38, 29, 32, 9, 32, 23, 47, 25, 6, 10, 3, 29, 1, 8, 39, 29, 26, 31, 11, 46, 36, 6, 4

4c) Trial of ACO with p = 50 and e = 0.4

In [33]:
ACO_BPP(50, 0.4, 2)

First generation: 
Fitness:    Best:  12001.5   Worst:  24004.0   Average:  16376.62
    Best path:  [21, 22, 36, 10, 40, 34, 40, 31, 20, 26, 49, 24, 32, 37, 45, 39, 2, 5, 20, 1, 34, 18, 7, 14, 16, 27, 4, 37, 23, 37, 42, 17, 41, 17, 36, 6, 3, 4, 23, 12, 42, 27, 50, 27, 11, 3, 50, 34, 15, 38, 16, 23, 32, 30, 23, 19, 13, 19, 36, 18, 16, 5, 25, 38, 21, 11, 17, 8, 1, 20, 36, 50, 33, 17, 22, 34, 29, 48, 39, 48, 33, 7, 19, 49, 21, 29, 29, 27, 39, 5, 8, 30, 24, 25, 41, 44, 32, 35, 50, 49, 40, 6, 47, 8, 42, 42, 6, 28, 26, 4, 41, 39, 6, 17, 45, 31, 24, 22, 1, 43, 11, 50, 11, 8, 37, 33, 26, 33, 3, 20, 43, 11, 36, 30, 3, 20, 23, 38, 17, 30, 32, 23, 46, 9, 3, 19, 23, 46, 20, 14, 3, 2, 36, 20, 45, 35, 4, 16, 20, 40, 39, 23, 12, 43, 31, 13, 18, 5, 31, 35, 22, 39, 32, 39, 40, 31, 21, 5, 25, 26, 17, 30, 16, 8, 43, 27, 1, 26, 18, 33, 7, 30, 33, 2, 11, 47, 6, 12, 36, 8, 40, 3, 29, 14, 33, 30, 11, 28, 20, 32, 27, 40, 14, 23, 19, 12, 7, 25, 44, 44, 44, 23, 28, 37, 19, 13, 4, 29, 7, 23, 49, 5, 31, 29, 8, 3

4d) Trial of ACO with p = 50 and e = 0.15

In [34]:
ACO_BPP(50, 0.15, 2)

First generation: 
Fitness:    Best:  11129.0   Worst:  19995.0   Average:  15608.26
    Best path:  [39, 38, 33, 41, 15, 14, 37, 48, 4, 38, 10, 22, 34, 36, 46, 30, 35, 23, 4, 20, 42, 39, 46, 15, 17, 24, 48, 26, 41, 17, 23, 14, 24, 35, 36, 2, 15, 34, 29, 30, 32, 13, 7, 12, 17, 29, 26, 1, 32, 12, 18, 47, 11, 31, 8, 40, 9, 29, 50, 37, 4, 44, 14, 4, 31, 5, 42, 1, 6, 2, 47, 29, 48, 24, 4, 34, 41, 41, 38, 31, 31, 18, 44, 23, 5, 9, 15, 48, 25, 31, 15, 12, 40, 35, 33, 21, 1, 9, 50, 49, 20, 49, 1, 5, 5, 1, 20, 33, 2, 14, 43, 1, 13, 9, 8, 1, 37, 8, 34, 24, 44, 27, 6, 12, 1, 16, 27, 24, 39, 7, 8, 10, 26, 40, 47, 19, 41, 16, 21, 2, 15, 5, 31, 27, 20, 18, 20, 36, 15, 3, 3, 29, 23, 17, 18, 17, 10, 16, 30, 1, 47, 47, 13, 11, 16, 30, 1, 34, 9, 28, 24, 3, 1, 44, 36, 27, 48, 14, 7, 3, 7, 45, 44, 29, 8, 29, 42, 4, 35, 45, 35, 33, 33, 4, 46, 9, 5, 49, 40, 19, 26, 49, 45, 37, 18, 33, 42, 43, 13, 48, 4, 19, 20, 23, 37, 13, 4, 36, 18, 6, 45, 15, 9, 35, 30, 24, 43, 32, 16, 18, 3, 41, 49, 10, 27, 20, 50, 27, 